In [1]:
import cv2
import numpy as np
from scipy import stats

In [2]:
PATH = 'pedestrian'
N = 60
BUFOR_SIZE = 30

In [3]:
f = open(f'{PATH}/temporalROI.txt','r') 
line = f.readline()
roi_start , roi_end = line.split() 
roi_start = int(roi_start) 
roi_end = int(roi_end)

I_start = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start)
HEIGHT, WIDTH, _ = I_start.shape

In [4]:
def gray_and_type(I):  # noqa: E741
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)  # noqa: E741
    I = I.astype('uint8')  # noqa: E741
    return I

def binarization(I, threshold=15):  # noqa: E741
    (_, thresh) = cv2.threshold(I, threshold,255,cv2.THRESH_BINARY)
    return thresh

def morphological_operations(B, kernel_size=3, blur_size=3, erode_iter=1, dilate_iter=2):
    kernel = np.ones((kernel_size, kernel_size),np.uint8)
    M = cv2.medianBlur(B, blur_size) 
    M = cv2.erode(M, kernel, iterations=erode_iter)
    M = cv2.dilate(M, kernel, iterations=dilate_iter)
    return M

def calculate_metrics(M, ref_mask, TP, TN, FP, FN):
    TP_M = np.logical_and((M==255), (ref_mask==255))
    TP = TP + np.sum(TP_M)

    TN_M = np.logical_and((M==0), (ref_mask==255))
    TN = TN + np.sum(TN_M)

    FP_M = np.logical_and((M==255), (ref_mask==0))
    FP = FP + np.sum(FP_M)

    FN_M = np.logical_and((M==0), (ref_mask==255))
    FN = FN + np.sum(FN_M)
    return TP, TN, FP, FN

In [5]:
TP, TN, FP, FN = 0, 0, 0, 0

buffer = np.zeros((BUFOR_SIZE, HEIGHT, WIDTH), dtype=np.uint8)

for index, i in enumerate(range(roi_start, roi_start + BUFOR_SIZE)):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i, cv2.IMREAD_GRAYSCALE)
    buffer[index] = img


BG = stats.mode(buffer, axis=0).mode[0]



for i in range(roi_start, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = gray_and_type(img)

    I_diff = cv2.absdiff(IG, BG).astype('uint8')

    B = binarization(I_diff, threshold=20)
    M = morphological_operations(B)

    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    BG = np.where(np.logical_and(IG > BG, M==0), BG + 1, BG)
    BG = np.where(np.logical_and(IG < BG, M==0), BG - 1, BG)


    TP, TN, FP, FN = calculate_metrics(M, ref_mask, TP, TN, FP, FN)

    cv2.imshow('original', img)
    cv2.imshow('operation', M)
    cv2.waitKey(3)



P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print(f'{np.median.__name__} precision: ', P, 'Recall: ', R, 'F1: ', F1)

/var/folders/z8/z_bh56_j7cxf4jpbcjf1hbmh0000gn/T/ipykernel_21718/4177547124.py:10: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  BG = stats.mode(buffer, axis=0).mode[0]


median precision:  0.932879822472304 Recall:  0.9756745798688851 F1:  0.9537974162458406
